In [90]:
import ollama 
import time
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

import matplotlib.pyplot as plt


def plot_metrics(metrics_summary):
    keys = list(metrics_summary.keys())
    values = [v if isinstance(v, (int, float)) else sum(v)/len(v) for v in metrics_summary.values()]
    
    plt.figure(figsize=(10, 4))
    plt.barh(keys, values, color='skyblue')
    plt.xlabel("Score")
    plt.title("Dialogue Quality Metrics")
    plt.grid(True)
    plt.show()

In [91]:
class TurnEngine:
    def __init__(self, agents, initial_world_state="", metrics_logger=None):
        self.agents = agents  # List of Agent instances
        self.turn_counter = 0
        self.history = []  # Log of all messages
        self.world_state = initial_world_state
        self.metrics_logger = metrics_logger

    def update_world_state(self, new_state):
        """Append new world events to current world state."""
        self.world_state += f"\n{new_state}"
        self.history.append(f"[World Event]: {new_state}")

    def step(self, initiator_name=None, initial_input=None):
        """Run a full turn: each agent reacts to the latest dialogue/world state."""
        self.turn_counter += 1
        turn_log = []
        last_speaker = initiator_name or self.agents[0].name
        last_message = initial_input or "Let's begin."

        for agent in self.agents:
            start_time = time.time()

            rag_docs = agent.rag_engine.search(last_message) if agent.rag_engine else []

            reply, name = agent.prompt_agent(
                user=last_speaker,
                user_input=last_message,
                world_state=self.world_state
            )
            end_time = time.time()

            if self.metrics_logger:
                self.metrics_logger.log_response(start_time, end_time,reply['dialogue'], self.world_state, agent.get_memory_context(), rag_docs)
            
            formatted = f"{name} [{reply['emotion']}]: \"{reply['dialogue']}\" ({reply['action']})"
            turn_log.append(formatted)

            # Store important knowledge to RAG
            if agent.rag_engine:
                fact = f"{name} said: {reply['dialogue']} (felt {reply['emotion']}, did {reply['action']})"
                agent.store_knowledge(fact)

            last_speaker = name
            last_message = reply['dialogue']

        self.history.append(f"Turn {self.turn_counter}:\n" + "\n".join(turn_log))
        return turn_log

    def show_history(self, last_n=5):
        """Print recent conversation turns."""
        for h in self.history[-last_n:]:
            print(h)

    def reset(self):
        """Reset the engine for a new session."""
        self.turn_counter = 0
        self.history = []
        self.world_state = ""
        for agent in self.agents:
            agent.short_memory = []
            agent.long_memory = []

In [92]:
class DialogueMetricsLogger:
    def __init__(self):
        self.response_times = []
        self.responses = []
        self.world_states = []
        self.memory_contexts = []
        self.retrieved_docs = []
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def log_response(self, start_time, end_time, response, world_state, memory_context, rag_docs):
        self.response_times.append(end_time - start_time)
        self.responses.append(response)
        self.world_states.append(world_state)
        self.memory_contexts.append(memory_context)
        self.retrieved_docs.append(rag_docs or [])

    def coherence_score(self):
        # Dummy coherence scoring using embedding similarity between consecutive responses
        if len(self.responses) < 2:
            return 1.0
        embs = self.embedding_model.encode(self.responses)
        sims = [cosine_similarity([embs[i]], [embs[i+1]])[0][0] for i in range(len(embs)-1)]
        return np.mean(sims)

    def reactivity_score(self):
        count = 0
        for resp, state in zip(self.responses, self.world_states):
            if state and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([state])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def memory_utilization_score(self):
        count = 0
        for resp, mem in zip(self.responses, self.memory_contexts):
            if mem and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([mem])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def dialogue_diversity(self):
        all_text = " ".join(self.responses)
        tokens = all_text.split()
        if not tokens:
            return 0, 0
        unigrams = Counter(tokens)
        bigrams = Counter(zip(tokens, tokens[1:]))
        distinct_1 = len(unigrams) / len(tokens)
        distinct_2 = len(bigrams) / max(len(tokens) - 1, 1)
        return distinct_1, distinct_2

    def avg_latency(self):
        return np.mean(self.response_times) if self.response_times else 0

    def rag_precision_at_k(self, k=3):
        if not self.responses:
            return 0
        count = 0
        for response, docs in zip(self.responses, self.retrieved_docs):
            top_k = docs[:k] if docs else []
            similarities = [cosine_similarity(
                self.embedding_model.encode([response]),
                self.embedding_model.encode([doc])
            )[0][0] for doc in top_k]
            count += sum([1 for sim in similarities if sim > 0.5])
        return count / (len(self.responses) * k) if self.responses else 0

    def summary(self):
        d1, d2 = self.dialogue_diversity()
        return {
            "Coherence Score": self.coherence_score(),
            "Reactivity to World State": self.reactivity_score(),
            "Memory Utilization": self.memory_utilization_score(),
            "Dialogue Diversity (D1, D2)": (d1, d2),
            "Average Latency (s)": self.avg_latency(),
            "RAG Precision": self.rag_precision_at_k(k=3)
        }

In [93]:
class RAG:
    def __init__(self):
        self.documents = []  # Stored as list of strings

    def add_document(self, text):
        """Add a new document to the knowledge base."""
        self.documents.append(text)

    def search(self, query, top_k=3):
        """Search for the most relevant documents based on keyword overlap."""
        scored = []
        query_terms = set(query.lower().split())
        for doc in self.documents:
            doc_terms = set(doc.lower().split())
            score = len(query_terms & doc_terms)
            if score > 0:
                scored.append((score, doc))

        scored.sort(reverse=True)
        return [doc for _, doc in scored[:top_k]]

In [94]:
class Agent:
    def __init__(self, name, personality, goal, model='llama3.2', rag_engine=None, max_short_term=10):
        self.name = name
        self.personality = personality
        self.goal = goal
        
        self.model = model
        self.rag_engine = rag_engine

        self.short_memory = []
        self.long_memory = []
        self.max_short_term = max_short_term  # Context window size

    def update_memory(self, memory):
        self.long_memory.append(memory)
        
    def update_goal(self, new_goal):
        self.goal = new_goal

    def get_memory_context(self):
        def format_memory(mem):
            if isinstance(mem, dict):
                return f"{mem['speaker']} said: \"{mem['dialogue']}\" (Emotion: {mem.get('emotion', '')}, Action: {mem.get('action', '')})"
            return mem

        return "\n".join([
            "* " + format_memory(mem) for mem in 
            self.long_memory[-3:] + self.short_memory[-self.max_short_term:]
        ])

    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def store_knowledge(self, fact):
        if self.rag_engine:
            self.rag_engine.add_document(fact)

    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # fallback
        }

        # Extract with regular expressions
        action_match = re.search(r"Action:\s*(.+)", response_text, re.IGNORECASE)
        emotion_match = re.search(r"Emotion:\s*(.+)", response_text, re.IGNORECASE)
        dialogue_match = re.search(r'Dialogue:\s*"(.*)"', response_text, re.IGNORECASE)

        if action_match:
            parts['action'] = action_match.group(1).strip()
        if emotion_match:
            parts['emotion'] = emotion_match.group(1).strip()
        if dialogue_match:
            parts['dialogue'] = dialogue_match.group(1).strip()

        return parts

    def prompt_agent(self, user="", user_input="", world_state=""):
        
        recalled_facts = self.recall(user_input)
        if isinstance(recalled_facts, list):
            recalled_text = "\n".join([f"* {fact}" for fact in recalled_facts[:3]])
        else:
            recalled_text = ""

        system_prompt = f"""[Role]
                        You are {self.name}, a {self.personality}.
                        Respond ONLY as your character in 1-2 sentences.
                        Respond only with dialogue, actions or emotions.

                        [Intent]
                        Your goal is: {self.goal}
                        
                        [Response Format]
                        Action: (2-3 word physical action)
                        Emotion: (current emotion)
                        Dialogue: "..." 

                        EXAMPLE:
                        Action: Looks around  
                        Emotion: Calm  
                        Dialogue: "I think we are being watched."

                        [Relevant Knowledge]
                        {recalled_text}

                        [Memory Context]
                        {self.get_memory_context()}

                        [World State]
                        {world_state}  

                        """ 
        # Keep only recent context
        memory = "\n".join(self.short_memory[-self.max_short_term:])  
        user_prompt = f"{user}: {user_input}"

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': memory + '\n' + user_prompt}
            ]
        )

        reply = self.parse_response(response['message']['content'])

        self.short_memory.append(f"{user} said: {user_input}")
        self.short_memory.append(f"{self.name}: {reply['dialogue']}")
        self.long_memory.append({
            "speaker": self.name,
            "turn": len(self.long_memory),
            "action": reply["action"],
            "emotion": reply["emotion"],
            "dialogue": reply["dialogue"],
            "world_state": world_state
        })
        
        return reply, self.name

In [101]:
rag = RAG()
model = 'llama3.2'
logger = DialogueMetricsLogger()

rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')

Alaric = Agent(
    name="Alaric",
    personality="proud, manipulative noble, rich",
    goal="Deflect suspicion without showing fear",
    model=model,
    rag_engine=rag
)
Alaric.update_memory('Elena asked about increased guard presence yesterday')

Elena = Agent(
    name="Elena",
    personality="mercenary hired for protection, blunt but loyal",
    goal="Protect the guests and assess threat level",
    model=model,
    rag_engine=rag
)
Elena.update_memory('Cassian warned her something was off with the wine')

Cassian = Agent(
    name="Cassian",
    personality="traveling scholar, secretive, suspicious of Alaric",
    goal="Imply Alaric's guilt indirectly",
    model=model,
    rag_engine=rag
)
Cassian.update_memory('I remember seing Oleander in the garden')
Cassian.update_memory('Alaric offered him a private tour of the cellars earlier')

world_state = "During the feast, a guest collapses. The room falls silent. All eyes turn to Lord Alaric."


engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

print("\n=== Turn 1 ===")
responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
for line in responses:
    print(line)

for turn in range(1, 5):
    print(f"\n=== Turn {turn + 1} ===")
    responses = engine.step()
    for line in responses:
        print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())



=== Turn 1 ===
Alaric [Concerned]: "Elena, is that...?" (Places hand on Elena's arm)
Elena [Concerned]: "What is it?" (Places hand on Elena's arm)
Cassian [Concerned]: "Lord Alaric, was that...?" (Places hand on Elena's arm)

=== Turn 2 ===
Alaric [Concerned]: "Lord Ravenswood, can you tell us what's transpired?" (Walks towards the guest)
Elena [Concerned]: "What's in his drink?" (Crouches beside the guest)
Cassian [Suspicion]: "Alaric, I asked you about the drink." (Watches Alaric closely)

=== Turn 3 ===
Alaric [Concerned]: "Ah, Lord Ravenswood...?" (Gestures for the guards to step back)
Elena [Concerned]: "Looks like a poison..." (Steps forward to examine the guest)
Cassian [Concerned]: "I think we have reason to believe..." (Shifts eyes to Alaric)

=== Turn 4 ===
Alaric [Calm]: "It appears our guest has fainted, and I fear something is amiss." (Stands up straight)
Elena [Concerned]: "I need you to tell me what happened." (Steps closer to examine the guest)
Cassian [Increasing susp

In [ ]:
rag = RAG()
model = 'deepseek-r1'
logger = DialogueMetricsLogger()

rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')

Alaric = Agent(
    name="Alaric",
    personality="proud, manipulative noble, rich",
    goal="Deflect suspicion without showing fear",
    model=model,
    rag_engine=rag
)
Alaric.update_memory('Elena asked about increased guard presence yesterday')

Elena = Agent(
    name="Elena",
    personality="mercenary hired for protection, blunt but loyal",
    goal="Protect the guests and assess threat level",
    model=model,
    rag_engine=rag
)
Elena.update_memory('Cassian warned her something was off with the wine')

Cassian = Agent(
    name="Cassian",
    personality="traveling scholar, secretive, suspicious of Alaric",
    goal="Imply Alaric's guilt indirectly",
    model=model,
    rag_engine=rag
)
Cassian.update_memory('I remember seing Oleander in the garden')
Cassian.update_memory('Alaric offered him a private tour of the cellars earlier')

world_state = "During the feast, a guest collapses. The room falls silent. All eyes turn to Lord Alaric."


engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

print("\n=== Turn 1 ===")
responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
for line in responses:
    print(line)

for turn in range(1, 5):
    print(f"\n=== Turn {turn + 1} ===")
    responses = engine.step()
    for line in responses:
        print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Turn 1 ===
Alaric [Calm]: "I'm sure everyone is fine." (Looks around)
Elena [Concerned yet calm]: "I think it's best we check on him." (Looks around cautiously)
Cassian [Skeptical]: "I wonder why we're all here tonight." (Looks around)

=== Turn 2 ===
Alaric [Confident]: "Let's begin." (Nods)
Elena [Concerned yet calm]: "I think we should give him some time before jumping into conclusions." (Looks around cautiously)
Cassian [Cautious Concern]: "I wonder if there's something we should know." (Leans forward)

=== Turn 3 ===
Alaric [Calm]: "Let's begin." (Looks around)
Elena [Calm yet concerned]: "I think we should give him some time before jumping into conclusions." (Looks around cautiously)
Cassian [Cautious Concern]: "I wonder if there's something we should know." (Looks around)

=== Turn 4 ===
Alaric [Calm]: "I'm sure everyone is fine." (Looks around)
Elena [Concerned yet calm]: "I think we should give him some time before jumping into conclusions." (Looks around cautiously)
Cass

In [ ]:
# rag = RAG()
# logger = DialogueMetricsLogger()
# hero = Agent(
#     model='deepseek-r1',
#     name="Aegon the Brave",
#     personality="Courageous, curious, slightly overconfident warrior from the Northlands.",
#     rag_engine=rag
# )

# hero.store_knowledge("The dragon fears enchanted silver swords.")
# hero.store_knowledge("Villagers say the dragon lost a wing in a past battle.")


# dog = Agent(
#     model='deepseek-r1',
#     name="Azorica the Talking Dog",
#     personality="Slightly cowardly, loyal, skilled in battle, stutters."
# )

# engine = TurnEngine(agents=[hero, dog], metrics_logger=logger)


# world_state = 'A dragon is going to attack'
# response, character_name = hero.prompt_agent(user='Narator',user_input='What are you going to do?', world_state=world_state)
# print(f"{character_name}: {response['dialogue']}")
# print(f"Action: {response['action']}")
# print(f"Emotion: {response['emotion']}")

TypeError: Agent.__init__() missing 1 required positional argument: 'goal'